In [1]:
%pip install -qU langchain-groq

Note: you may need to restart the kernel to use updated packages.


In [3]:
import os

os.environ["GROQ_API_KEY"] = "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"

from langchain_groq import ChatGroq

llm = ChatGroq(model="llama3-70b-8192",groq_api_key=os.environ.get("GROQ_API_KEY"),temperature=0.7)

In [5]:
%pip install langchain langchain_community langchain_chroma

  Using cached langchain_core-0.1.52-py3-none-any.whl.metadata (5.9 kB)
Using cached langchain_core-0.1.52-py3-none-any.whl (302 kB)
  Attempting uninstall: langchain-core
    Found existing installation: langchain-core 0.2.11
    Uninstalling langchain-core-0.2.11:
      Successfully uninstalled langchain-core-0.2.11
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
langchain-groq 0.1.6 requires langchain-core<0.3,>=0.2.2, but you have langchain-core 0.1.52 which is incompatible.
Note: you may need to restart the kernel to use updated packages.


In [9]:
%pip install langchainhub

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Note: you may need to restart the kernel to use updated packages.


In [14]:
%pip install pymupdf

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 25.7 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 14.9/14.9 MB 33.4 MB/s eta 0:00:0000:0100:01
Note: you may need to restart the kernel to use updated packages.


In [26]:
import bs4
from langchain import hub
from langchain_chroma import Chroma
from langchain_community.document_loaders import WebBaseLoader
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_text_splitters import RecursiveCharacterTextSplitter
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Chroma
from langchain.embeddings import SentenceTransformerEmbeddings
from langchain_community.document_loaders import PyMuPDFLoader



model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')
# Use PyMuPDFLoader to load the PDF
loader = PyMuPDFLoader("https://www.dbs-sar.com/LPB/Introduction.pdf")
docs = loader.load()

text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What is a lost person?")

/Library/Frameworks/Python.framework/Versions/3.12/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


'A lost person is someone who is unable to find their way or is separated from their group or family, often experiencing feelings of fear, dread, and embarrassment. This can include a child who has wandered away from their parents or a hiker who has become disoriented in the wilderness. The definition of "lost" can vary depending on the context and situation.'

In [29]:
rag_chain.invoke("Where was a a missing person with dementia found before?")

'The missing person with dementia was found across the road from the first pond, near a second pond, next to a drainage, in a briar patch.'

In [ ]:
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyMuPDFLoader
from langchain.embeddings import SentenceTransformerEmbeddings
from sentence_transformers import SentenceTransformer


# Define the directory to persist the vector store
persist_directory = './vectorstore'

# Load the SentenceTransformer model and embeddings
model = SentenceTransformer('all-MiniLM-L6-v2')
embeddings = SentenceTransformerEmbeddings(model_name='all-MiniLM-L6-v2')

loader = PyMuPDFLoader("https://www.dbs-sar.com/LPB/Introduction.pdf")
docs = loader.load()

# Split the documents into smaller chunks
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
splits = text_splitter.split_documents(docs)

# Create and save the vector store
vectorstore = Chroma.from_documents(documents=splits, embedding=embeddings, persist_directory=persist_directory)


In [34]:
import os
from langchain import hub
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain.vectorstores import Chroma

persist_directory = './vectorstore'
vectorstore = Chroma(embedding_function=embeddings, persist_directory=persist_directory)

# Retrieve and generate using the relevant snippets of the blog
retriever = vectorstore.as_retriever()
prompt = hub.pull("rlm/rag-prompt")

def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)

rag_chain = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

# Example query
rag_chain.invoke("where might we find a person with dementia?")

"Based on the context, a person with dementia might be found in an assisted living facility, as shown in Case Two, where a 60-year-old male with mild Alzheimer's disease was reported missing. They might also be found in a nearby outdoor area, such as a pond or a briar patch, as was the case in this scenario."